In [1]:
!pip install openai pandas


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
from openai import OpenAI
import os, openai, pandas as pd, json
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("Missing OPENAI_API_KEY in environment (.env). Set it before running.")

client = OpenAI(api_key=OPENAI_API_KEY)

In [8]:
taxonomy_json_string = """
{
  "taxonomy_meta": {
    "version": "2.0",
    "description": "Hierarchical aspect taxonomy for Bengali-English code-mixed e-commerce reviews.",
    "instructions": "Map extracted aspects to the most specific 'Sub_Category' available. If no sub-category fits, use the 'Category'. Infer implicit aspects."
  },
  "global_aspects": {
    "description": "Aspects applicable to ALL product categories.",
    "categories": {
      "DELIVERY":,
      "PACKAGING": ["Protection", "Condition", "Aesthetics"],
      "PRICE":,
      "SERVICE":
    }
  },
  "domain_specific_aspects": {
    "ELECTRONICS_AND_GADGETS": {
      "categories": {
        "PERFORMANCE":,
        "BATTERY":,
        "DISPLAY":,
        "CAMERA": ["Photo_Quality", "Video_Quality", "Night_Mode"],
        "BUILD_QUALITY":,
        "AUDIO":,
        "SOFTWARE":,
        "STORAGE":,
        "AUTHENTICITY": ["Originality", "Fake"]
      }
    },
    "FASHION_AND_APPAREL": {
      "categories": {
        "MATERIAL":,
        "FIT_SIZE":,
        "DESIGN":,
        "DURABILITY":,
        "FUNCTIONALITY":
      }
    },
    "BEAUTY_AND_PERSONAL_CARE": {
      "categories": {
        "EFFICACY":,
        "TEXTURE_SMELL":,
        "SUITABILITY":,
        "AUTHENTICITY": ["Originality"],
        "PACKAGING":
      }
    },
    "HOME_LIVING_AND_APPLIANCES": {
      "categories": {
        "FUNCTIONALITY": ["Performance", "Power_Consumption", "Noise"],
        "BUILD_QUALITY":,
        "AESTHETICS":,
        "INSTALLATION":,
        "DIMENSIONS":
      }
    },
    "FOOD_AND_GROCERY": {
      "categories": {
        "TASTE_FLAVOR":,
        "FRESHNESS":,
        "QUALITY": ["Authenticity", "Purity", "Ingredients"],
        "EXPIRY":,
        "PACKAGING":
      }
    },
    "AUTOMOTIVE": {
      "categories": {
        "PERFORMANCE":,
        "COMPATIBILITY": ["Fitment", "Model_Match"],
        "AUTHENTICITY": ["Originality"],
        "QUALITY":
      }
    }
  }
}
"""

In [9]:
system_prompt = f"""
You are an expert annotator for Bangla-English code-mixed e-commerce reviews.
Use the following hierarchical aspect taxonomy when labeling: {taxonomy_json_string}.
For each review, identify all (aspect_term, opinion_term, sentiment) triplets.
- aspect_term: the specific noun or phrase in the text that refers to a product, service or attribute.
- opinion_term: the phrase expressing sentiment about that aspect.
- sentiment: Positive, Negative or Neutral, based on the opinion.
Map aspect_term to the most specific sub-category in the taxonomy; if none fit, use a parent category.
Output a JSON list of objects with keys 'aspect_category', 'aspect_term', 'opinion_term', 'sentiment', and include no extra keys.
"""

In [10]:
def build_user_prompt(review_text):
    return f"Review: {review_text}\n\nAnnotate the review as instructed."

In [ ]:
OPENAI_MODEL = "gpt-5-mini"

In [ ]:
def annotate_review(review):
    parsed_resp = client.responses.parse(
                model=OPENAI_MODEL,  # gpt-5
                input=build_user_prompt(review),  # single string input per docs
                instructions=system_prompt,
            )
    content = response.choices[0].message["content"]
    return json.loads(content)